In [20]:
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras import optimizers
from keras.layers.advanced_activations import ELU, PReLU, LeakyReLU
from keras.layers import Dense, Dropout, Activation, Flatten

In [21]:
model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(513, 800, 3)))
model.add(Activation('relu'))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(10))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(57))
model.add(Activation('softmax'))

model.summary()

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 513, 800, 8)       224       
_________________________________________________________________
activation_7 (Activation)    (None, 513, 800, 8)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 511, 798, 8)       584       
_________________________________________________________________
activation_8 (Activation)    (None, 511, 798, 8)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 255, 399, 8)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 255, 399, 8)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 255, 399, 16)     

In [23]:
model.load_weights('F:/voice-classification-master/scripts/demo/my_model_weights.h5')


In [24]:
import subprocess
import matplotlib.pyplot
import os
import matplotlib.pyplot as plt
rootdir = 'F:\\voice-classification-master\\lang_example_90\\train\\'

In [25]:
spectograms = []
spect_read = []
spectograms_ids = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith('png'): 
            try:
                x = plt.imread(subdir+'/'+file)
            except:
                continue
            if str(x.shape) == '(513, 800, 3)': 
                spect_read.append(x)
                #print(subdir) 
                name = subdir.replace(rootdir, '')
                #print(name)
                #name = name.replace('/spects', "")
                spectograms_ids.append(name)
                spectograms.append(file)
x_train = spect_read
y_train = spectograms_ids
y_train

['hamza',
 'hamza',
 'hamza',
 'hamza',
 'hamza',
 'hamza',
 'isabel',
 'isabel',
 'isabel',
 'isabel',
 'isabel',
 'isabel',
 'kais',
 'kais',
 'kais',
 'kais',
 'kais',
 'kais',
 'yousef',
 'yousef',
 'yousef',
 'yousef',
 'yousef',
 'yousef']

In [26]:
rootdir = 'F:\\voice-classification-master\\lang_example_90\\test\\'
spectograms = []
spect_read = []
spectograms_ids = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith('png'): 
            try:
                x = plt.imread(subdir+'/'+file)
            except:
                continue
            if str(x.shape) == '(513, 800, 3)': 
                spect_read.append(x)
                name = subdir.replace(rootdir, '')
                #name = name.replace('/spects', "")
                spectograms_ids.append(name)
                spectograms.append(file)
x_test = spect_read
y_test = spectograms_ids
y_test

['hamza',
 'hamza',
 'hamza',
 'isabel',
 'isabel',
 'isabel',
 'isabel',
 'kais',
 'kais',
 'kais',
 'kais',
 'yousef',
 'yousef',
 'yousef',
 'yousef']

In [27]:

import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
y_temp_train = y_train
encoder.fit(y_temp_train)
encoded_Y = encoder.transform(y_temp_train)
dummy_y = np_utils.to_categorical(encoded_Y)

y_temp2_train = y_test
encoder.fit(y_temp2_train)
encoded_Y = encoder.transform(y_temp2_train)
dummy2_y = np_utils.to_categorical(encoded_Y)

In [28]:
dummy2_y

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [30]:
from keras.models import Model
model2 = Model(inputs=model.input, outputs=model.get_layer('flatten_2').output)

In [31]:
svm_x_train = []
svm_y_train = []
for i in range(len(x_train)):
    x_1 = np.expand_dims(x_train[i], axis=0)
    flatten_2_features = model2.predict(x_1)
    svm_x_train.append(flatten_2_features)
    svm_y_train.append(dummy_y[i])

In [32]:
svm_y_train

[array([1., 0., 0., 0.], dtype=float32),
 array([1., 0., 0., 0.], dtype=float32),
 array([1., 0., 0., 0.], dtype=float32),
 array([1., 0., 0., 0.], dtype=float32),
 array([1., 0., 0., 0.], dtype=float32),
 array([1., 0., 0., 0.], dtype=float32),
 array([0., 1., 0., 0.], dtype=float32),
 array([0., 1., 0., 0.], dtype=float32),
 array([0., 1., 0., 0.], dtype=float32),
 array([0., 1., 0., 0.], dtype=float32),
 array([0., 1., 0., 0.], dtype=float32),
 array([0., 1., 0., 0.], dtype=float32),
 array([0., 0., 1., 0.], dtype=float32),
 array([0., 0., 1., 0.], dtype=float32),
 array([0., 0., 1., 0.], dtype=float32),
 array([0., 0., 1., 0.], dtype=float32),
 array([0., 0., 1., 0.], dtype=float32),
 array([0., 0., 1., 0.], dtype=float32),
 array([0., 0., 0., 1.], dtype=float32),
 array([0., 0., 0., 1.], dtype=float32),
 array([0., 0., 0., 1.], dtype=float32),
 array([0., 0., 0., 1.], dtype=float32),
 array([0., 0., 0., 1.], dtype=float32),
 array([0., 0., 0., 1.], dtype=float32)]

In [33]:
from sklearn import svm
svm_x_train = np.array(svm_x_train)
clf = svm.SVC(kernel='rbf', class_weight='balanced')
dataset_size = len(svm_x_train)
svm_x_train = np.array(svm_x_train).reshape(dataset_size,-1)
svm_y_train = np.array(svm_y_train)
svm_y_train = [np.where(r==1)[0][0] for r in svm_y_train]

In [34]:
clf.fit(svm_x_train, svm_y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [35]:
svm_x_test = []
svm_y_test = []
for i in range(len(x_test)):
    x_1 = np.expand_dims(x_test[i], axis=0)
    #x_1 = preprocess_input(x_1)
    flatten_2_features = model2.predict(x_1)
    svm_x_test.append(flatten_2_features)
    svm_y_test.append(dummy2_y[i])
svm_x_test = np.array(svm_x_test)

In [36]:
dataset_size = len(svm_x_test)
svm_x_test = np.array(svm_x_test).reshape(dataset_size,-1)
svm_y_test = [np.where(r==1)[0][0] for r in svm_y_test]

In [37]:
print(svm_y_test)

[0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]


In [38]:
print(clf.predict(svm_x_test))

[0 1 0 1 1 1 1 2 2 2 2 3 3 3 3]


In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(svm_y_test, clf.predict(svm_x_test))

0.9333333333333333

In [48]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf, svm_x_test, svm_y_test, cv=4)


C:\Users\laadouz\Anaconda3\envs\tensorflow-session\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


array([0.75, 0.75, 0.5 , 1.  ])